# Agents
Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user input. In these types of chains, there is a "agent" (backed by an LLM) which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

When used correctly agents can be extremely powerful. The purpose of this notebook is to show you how to easily use agents through the simplest, highest level API. If you want more low level control over various components, check out the documentation for custom agents (coming soon).

## Concepts

In order to understand agents, you should understand the following concepts:
- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- LLM: The language model powering the agent.
- AgentType: The type of agent to use. This should be a string. For a list of supported agents, see here. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

## Tools
When constructing your own agent, you will need to provide it with a list of Tools that it can use. A Tool is defined as below.

```python
class Tool(NamedTuple):
    """Interface for tools."""

    name: str
    func: Callable[[str], str]
    description: Optional[str] = None
```

The two required components of a Tool are the name and then the tool itself. A tool description is optional, as it is needed for some agents but not all.

## Loading an agent


In [1]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [2]:
# Load the tool configs that are needed.
from langchain import LLMMathChain, SerpAPIChain
llm = OpenAI(temperature=0)
search = SerpAPIChain()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [5]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, verbose=True)

In [6]:
agent.run("What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?")

What is the age of Olivia Wilde's boyfriend raised to the 0.23 power?
Thought: I need to find the age of Olivia Wilde's boyfriend
Action: Search
Action Input: "Olivia Wilde's boyfriend"
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find the age of Harry Styles
Action: Search
Action Input: "Harry Styles age"
Observation: 28 years
Thought: I need to calculate 28 to the 0.23 power
Action: Calculator
Action Input: 28^0.23

> Entering new chain...
28^0.23

```python
print(28**0.23)
```

Answer: 2.1520202182226886

> Finished chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer
Final Answer: 2.1520202182226886

'2.1520202182226886'